<a href="https://colab.research.google.com/github/BAMBAMGit/baseball/blob/main/baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
from numpy.lib.stride_tricks import broadcast_shapes
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
DataTable.max_columns = 200

df = pd.read_csv("playing-2021.csv", on_bad_lines="skip")   # from chadwickbureau  ?retrosplits .. daybyday data   playing-2021.csv is 21MB file which has every game that year
game_key_as_date = df['game.key'].str[3:].astype(int)

# 1st 1/3 of season -- 4/1/21 to 5/31/21 -- Get a baseline for every player
# 2nd 1/3 of season -- 6/1/21 to 7/31/21 -- Train
# 3rd 1/3 of season -- 8/1/21 to 9/30/21 -- Test


df1 = df[game_key_as_date < 202106010]
df2 = df[(game_key_as_date > 202105310) & (game_key_as_date < 202108010)]
df3 = df[game_key_as_date > 202107310]

df1 = df1.sort_values(['team.key','person.key','game.date'])
df1 = df1[df1['B_AB']>0]

df1['OB%'] = ((df1["B_H"] + df1["B_BB"] + df1["B_IBB"] + df1["B_HP"]) /(df1["B_AB"])).round(decimals=3)

gameKey_column = df1.pop('game.key')
df1.insert(0, 'game.key', gameKey_column)
teamKey_column = df1.pop('team.key')
df1.insert(1, 'team.key', teamKey_column)
slot_column = df1.pop('slot')
df1.insert(2, 'slot', slot_column)
personKey_column = df1.pop('person.key')
df1.insert(3, 'person.key', personKey_column)
OBP_column = df1.pop('OB%')
df1.insert(4, 'OB%', OBP_column)

dftest = df1
dftest2 = df1['person.key']
player_list = dftest2.unique()

#reset indices
dftest = dftest.reset_index(drop=True)
dftest

dftest.loc[0,'Running_OB%'] = dftest.loc[0,'OB%']

count = 1

for x in player_list:
  for i in range(1,len(dftest)):
    if (dftest.loc[i-1,'person.key'] != x) & (dftest.loc[i,'person.key'] == x):
      dftest.loc[i,'Running_OB%'] = dftest.loc[i,'OB%']
      count = 1
    elif dftest.loc[i,'person.key'] == x:
      count = count + 1
      dftest.loc[i,'Running_OB%'] = (dftest.loc[i-1,'Running_OB%']*(count-1) + dftest.loc[i,'OB%']) / count


Running_OBP_column = dftest.pop('Running_OB%')
dftest.insert(5, 'Running_OB%', Running_OBP_column)
dftest['Running_OB%'] = dftest['Running_OB%'].round(decimals=3)


dftest.head(300)
# for ind in dftest.index:
#      dftest['Name'][ind]
#      dftest['Stream'][ind]



# len(df1['game.key'].unique())    ===> 800 unique games
# df.columns.values.tolist()

# https://github.com/droher/boxball/blob/master/transform/src/schemas/retrosheet.py    explanation of all columns


,game.key,team.key,slot,person.key,OB%,Running_OB%,game.source,game.date,game.number,appear.date,...,F_P_POS,F_P_G,F_P_GS,F_P_OUT,F_P_TC,F_P_PO,F_P_A,F_P_E,F_P_DP,F_P_TP
0,KCA202104140,ANA,8,bemba001,0.250,0.250,evt,2021-04-14,0,2021-04-14,...,0,0,0,0,0,0,0,0,0,0
1,ANA202104210,ANA,8,bemba001,0.333,0.292,evt,2021-04-21,0,2021-04-21,...,0,0,0,0,0,0,0,0,0,0
2,HOU202104230,ANA,9,bemba001,0.250,0.278,evt,2021-04-23,0,2021-04-23,...,0,0,0,0,0,0,0,0,0,0
3,HOU202104240,ANA,3,bemba001,0.000,0.208,evt,2021-04-24,0,2021-04-24,...,1,1,0,3,0,0,0,0,0,0
4,HOU202104250,ANA,9,bemba001,0.250,0.217,evt,2021-04-25,0,2021-04-25,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,ANA202105090,ANA,6,rojaj002,0.000,0.252,evt,2021-05-09,0,2021-05-09,...,0,0,0,0,0,0,0,0,0,0
296,HOU202105100,ANA,6,rojaj002,0.500,0.263,evt,2021-05-10,0,2021-05-10,...,0,0,0,0,0,0,0,0,0,0
297,HOU202105110,ANA,6,rojaj002,0.000,0.252,evt,2021-05-11,0,2021-05-11,...,0,0,0,0,0,0,0,0,0,0
298,HOU202105120,ANA,5,rojaj002,0.250,0.251,evt,2021-05-12,0,2021-05-12,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from re import X
#@title Batting
df = pd.read_csv("Batting.csv")
df['OB%'] = (df['H']+df['BB']+df['IBB']+df['HBP'])/df['AB']

x = df['yearID'] == 2021
y = df['OB%'] > 0.0001
z = df['teamID'] == 'NYA'

df[x].sort_values(['teamID'],['AB'])

# top15 = df[x & y & z].sort_values(by=['AB'], ascending=False).head(15)
# top15['OB%'].mean()

#len(df[df['yearID'] == 2021])
#df[(df["teamID"] == "NYA") & (df["yearID"] == 2021)]
#df.loc[2955:2984,["AB"]].sum()

In [ ]:
#@title Pitching
df = pd.read_csv("Pitching.csv")
df["SO%"] = (df["SO"]/df["BFP"])
df["H%"] = (df["H"]/df["BFP"])
df["OB%"] = ((df["H"]+df["BB"])/df["BFP"])
df[(df["teamID"]=="LAN") & (df["yearID"]==2021) & (df["G"] > 40)].sort_values(by="OB%").round(decimals=2)